In [1]:
# !wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
# !unzip uncased_L-12_H-768_A-12.zip

In [2]:
BERT_VOCAB = 'uncased_L-12_H-768_A-12/vocab.txt'
BERT_INIT_CHKPNT = 'uncased_L-12_H-768_A-12/bert_model.ckpt'
BERT_CONFIG = 'uncased_L-12_H-768_A-12/bert_config.json'

In [3]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling
import tensorflow as tf
import numpy as np

/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

/home/husein/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorboard/compat/ten

In [4]:
tokenization.validate_case_matches_checkpoint(True,BERT_INIT_CHKPNT)
tokenizer = tokenization.FullTokenizer(
      vocab_file=BERT_VOCAB, do_lower_case=True)

In [5]:
text = 'A politician is a person active in party politics, or a person holding or seeking office in government. Politicians propose, support and create laws or policies that govern the land and, by extension, its people'
text

'A politician is a person active in party politics, or a person holding or seeking office in government. Politicians propose, support and create laws or policies that govern the land and, by extension, its people'

In [6]:
aug_percent = 0.8
splitted = text.split()
size = len(splitted)
cnt = int(aug_percent * size)
cnt

28

In [7]:
# !wget https://raw.githubusercontent.com/6/stopwords-json/master/dist/en.json

In [8]:
import json
with open('en.json') as fopen:
    stopwords = json.load(fopen)

In [9]:
import random
import string

results = []
samples = random.sample([i for i in range(size)], cnt)
for token_idx, token in enumerate(samples):
    if splitted[token] in string.punctuation:
        continue
    if splitted[token] in stopwords:
        continue
    results.append(token)
    
results

[34, 20, 8, 17, 18, 4, 29, 14, 1, 32, 0, 15, 7, 25, 5, 11, 22, 27]

In [10]:
import unicodedata

def whitespace_tokenize(text):
    text = text.strip()
    if not text:
        return []
    tokens = text.split()
    return tokens

class BasicTokenizer(object):

    def __init__(self, do_lower_case=True, never_split=None):
        if never_split is None:
            never_split = []
        self.do_lower_case = do_lower_case
        self.never_split = never_split

    def tokenize(self, text, never_split=None):
        never_split = self.never_split + (never_split if never_split is not None else [])
        text = self._clean_text(text)
        orig_tokens = whitespace_tokenize(text)
        split_tokens = []
        for token in orig_tokens:
            if token not in never_split:
                if self.do_lower_case:
                    token = token.lower()
                token = self._run_strip_accents(token)
                split_tokens.extend(self._run_split_on_punc(token))
            else:
                split_tokens.append(token)

        output_tokens = whitespace_tokenize(" ".join(split_tokens))
        return output_tokens

    def _run_strip_accents(self, text):
        text = unicodedata.normalize("NFD", text)
        output = []
        for char in text:
            cat = unicodedata.category(char)
            if cat == "Mn":
                continue
            output.append(char)
        return "".join(output)

    def _run_split_on_punc(self, text, never_split=None):
        if never_split is not None and text in never_split:
            return [text]
        chars = list(text)
        i = 0
        start_new_word = True
        output = []
        while i < len(chars):
            char = chars[i]
            if _is_punctuation(char):
                output.append([char])
                start_new_word = True
            else:
                if start_new_word:
                    output.append([])
                start_new_word = False
                output[-1].append(char)
            i += 1

        return ["".join(x) for x in output]
    
    def _clean_text(self, text):
        output = []
        for char in text:
            cp = ord(char)
            if cp == 0 or cp == 0xfffd or _is_control(char):
                continue
            if _is_whitespace(char):
                output.append(" ")
            else:
                output.append(char)
        return "".join(output)
    
def _is_control(char):
    if char == "\t" or char == "\n" or char == "\r":
        return False
    cat = unicodedata.category(char)
    if cat.startswith("C"):
        return True
    return False

def _is_whitespace(char):
    if char == " " or char == "\t" or char == "\n" or char == "\r":
        return True
    cat = unicodedata.category(char)
    if cat == "Zs":
        return True
    return False

def _is_punctuation(char):
    cp = ord(char)
    if ((cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or
            (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126)):
        return True
    cat = unicodedata.category(char)
    if cat.startswith("P"):
        return True
    return False

In [11]:
from bert.tokenization import WordpieceTokenizer, load_vocab, convert_by_vocab

class FullTokenizer(object):
    def __init__(self, vocab_file, do_lower_case=True):
        self.vocab = load_vocab(vocab_file)
        self.inv_vocab = {v: k for k, v in self.vocab.items()}
        self.basic_tokenizer = BasicTokenizer(do_lower_case=do_lower_case, 
                                              never_split = ['[CLS]', '[MASK]', '[SEP]'])
        self.wordpiece_tokenizer = WordpieceTokenizer(vocab=self.vocab)

    def tokenize(self, text):
        split_tokens = []
        for token in self.basic_tokenizer.tokenize(text):
            for sub_token in self.wordpiece_tokenizer.tokenize(token):
                split_tokens.append(sub_token)

        return split_tokens

    def convert_tokens_to_ids(self, tokens):
        return convert_by_vocab(self.vocab, tokens)

    def convert_ids_to_tokens(self, ids):
        return convert_by_vocab(self.inv_vocab, ids)

In [12]:
tokenizer = FullTokenizer(vocab_file=BERT_VOCAB, do_lower_case=True)

In [13]:
tokenizer.tokenize('[CLS] i like to eat husein [MASK] [SEP]')

['[CLS]', 'i', 'like', 'to', 'eat', 'hu', '##sei', '##n', '[MASK]', '[SEP]']

In [14]:
maskeds, indices = [], []

for index in results:
    new = splitted[:]
    new[index] = '[MASK]'
    tokens = tokenizer.tokenize(' '.join(new))
    new = ["[CLS]"] + tokens + ["[SEP]"]
    mask_index = new.index('[MASK]')
    masked_ids = tokenizer.convert_tokens_to_ids(new)
    maskeds.append(masked_ids)
    indices.append(mask_index)

In [15]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)

In [16]:
def top_k_logits(logits, k):
    if k == 0:
        return logits

    def _top_k():
        values, _ = tf.nn.top_k(logits, k=k)
        min_values = values[:, -1, tf.newaxis]
        return tf.where(
            logits < min_values,
            tf.ones_like(logits, dtype=logits.dtype) * -1e10,
            logits,
        )
    return tf.cond(
        tf.equal(k, 0),
        lambda: logits,
        lambda: _top_k(),
    )


def top_p_logits(logits, p):
    with tf.variable_scope('top_p_logits'):
        logits_sort = tf.sort(logits, direction='DESCENDING')
        probs_sort = tf.nn.softmax(logits_sort)
        probs_sums = tf.cumsum(probs_sort, axis=1, exclusive=True)
        logits_masked = tf.where(probs_sums < p, logits_sort, tf.ones_like(
            logits_sort)*1000)  # [batchsize, vocab]
        min_logits = tf.reduce_min(logits_masked, axis=1, keepdims=True)  # [batchsize, 1]
        return tf.where(
            logits < min_logits,
            tf.ones_like(logits, dtype=logits.dtype) * -1e10,
            logits,
        )


class Model:
    def __init__(
        self,
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.top_p = tf.placeholder(tf.float32, None)
        self.top_k = tf.placeholder(tf.int32, None)
        self.k = tf.placeholder(tf.int32, None)
        self.temperature = tf.placeholder(tf.float32, None)
        self.indices = tf.placeholder(tf.int32, [None, None])
        
        model = modeling.BertModel(
            config=bert_config,
            is_training=False,
            input_ids=self.X,
            use_one_hot_embeddings=False)
        
        output_layer = model.get_sequence_output()
        embedding = model.get_embedding_table()
        
        with tf.variable_scope('cls/predictions'):
            with tf.variable_scope('transform'):
                input_tensor = tf.layers.dense(
                    output_layer,
                    units = bert_config.hidden_size,
                    activation = modeling.get_activation(bert_config.hidden_act),
                    kernel_initializer = modeling.create_initializer(
                        bert_config.initializer_range
                    ),
                )
                input_tensor = modeling.layer_norm(input_tensor)
            
            output_bias = tf.get_variable(
            'output_bias',
            shape = [bert_config.vocab_size],
            initializer = tf.zeros_initializer(),
            )
            logits = tf.matmul(input_tensor, embedding, transpose_b = True)
            self.logits = tf.nn.bias_add(logits, output_bias)
            
            
        logits = tf.gather_nd(self.logits, self.indices)
        logits = logits / self.temperature
        
        def necleus():
            return top_p_logits(logits, self.top_p)
        
        def select_k():
            return top_k_logits(logits, self.top_k)
        
        logits = tf.cond(self.top_p > 0, necleus, select_k)
        self.samples = tf.multinomial(
                logits, num_samples=self.k, output_dtype=tf.int32)

In [17]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model()

sess.run(tf.global_variables_initializer())
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'bert')




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categorical` instead.


In [18]:
cls = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'cls')
cls

[<tf.Variable 'cls/predictions/transform/dense/kernel:0' shape=(768, 768) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/transform/dense/bias:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/transform/LayerNorm/beta:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/transform/LayerNorm/gamma:0' shape=(768,) dtype=float32_ref>,
 <tf.Variable 'cls/predictions/output_bias:0' shape=(30522,) dtype=float32_ref>]

In [19]:
saver = tf.train.Saver(var_list = var_lists + cls)
saver.restore(sess, BERT_INIT_CHKPNT)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from uncased_L-12_H-768_A-12/bert_model.ckpt


In [20]:
masked_padded = tf.keras.preprocessing.sequence.pad_sequences(maskeds,padding='post')
masked_padded.shape

(18, 42)

In [21]:
batch_indices = np.array([np.arange(len(indices)), indices]).T
batch_indices.shape

(18, 2)

In [22]:
samples = sess.run(model.samples, feed_dict = {model.X: masked_padded,
                                    model.top_p: 0.8,
                                    model.top_k: 100,
                                    model.temperature: 0.8,
                                    model.indices: batch_indices,
                                    model.k: 5})

In [23]:
for i in range(samples.shape[1]):
    print('SAMPLE %d'%(i))
    sample_i = samples[:, i]
    samples_tokens = tokenizer.convert_ids_to_tokens(sample_i)
    new_splitted = splitted[:]
    for no, index in enumerate(results):
        new_splitted[index] = samples_tokens[no]

    new = ' '.join(new_splitted)
    print('BEFORE:', text)
    print('AFTER:', new)
    print()

SAMPLE 0
BEFORE: A politician is a person active in party politics, or a person holding or seeking office in government. Politicians propose, support and create laws or policies that govern the land and, by extension, its people
AFTER: a politician is a person involved in national politics or a person holding or holding office in which they propose, propose and implement laws or regulations that protect the country and, by , its .

SAMPLE 1
BEFORE: A politician is a person active in party politics, or a person holding or seeking office in government. Politicians propose, support and create laws or policies that govern the land and, by extension, its people
AFTER: a politician is a person involved in local politics or a person holding or holding office in which they propose, propose and implement laws or regulations that protect the country and, by protecting its .

SAMPLE 2
BEFORE: A politician is a person active in party politics, or a person holding or seeking office in government. P